In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import re
import time
import urllib
import urllib.request
from matplotlib.gridspec import GridSpec
from os import listdir
import imagesize
import json

In [13]:
def text_ratio(img_path):
    img_path = f"./covers/text/{img_path}"
    image = cv2.imread(img_path, 0)
    text = cv2.countNonZero(image)
    size = image.size
    ratio = 1 - text/size
    return ratio

In [14]:
def get_image_dims(img_path):
    img_path = f"./covers/text/{img_path}"
    width, height = imagesize.get(img_path)
    return [width, height]

In [23]:
def get_contours_coordinates(path):
    image_path = '/Users/jessicacarr/Documents/J/github/magazine_text/covers/text/' + path
    image = cv2.imread(image_path)  # Corrected variable name to 'image'
    image_width = image.shape[1]
    image_height = image.shape[0]

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    bounding_rectangles = []

    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
    
        if area > 100:  # Adjust this threshold as needed
            x, y, w, h = cv2.boundingRect(contour)
        
            x_pct = (x / image_width) * 100
            y_pct = (y / image_height) * 100
            w_pct = (w / image_width) * 100
            h_pct = (h / image_height) * 100
        
            bounding_rectangles.append({
                "x": x, "x_pct": x_pct,
                "y": y, "y_pct": y_pct,
                "w": w, "w_pct": w_pct,
                "h": h, "h_pct": h_pct
            })

    return(bounding_rectangles)

In [15]:
files= [f for f in listdir("./covers/text/") if f.endswith(".png")]
df = pd.DataFrame(files, columns=['img_path'])
df.head()

,img_path
0,20161201-01.png
1,20211201-01.png
2,20120601-01.png
3,20120701-01.png
4,20180101-01.png


In [16]:
df.loc[:, 'dimensions'] = df.apply(lambda row : get_image_dims(row['img_path']), axis=1)
df[['width','height']] = pd.DataFrame(df.dimensions.tolist(), index= df.index)

In [17]:
df = df.drop('dimensions', axis=1)
df.loc[:,"wh_ratio"] = df.apply(lambda row : row['width'] / row['height'], axis=1)
df.head()

,img_path,width,height,wh_ratio
0,20161201-01.png,2480,3382,0.733294
1,20211201-01.png,2481,3383,0.733373
2,20120601-01.png,2480,3425,0.724088
3,20120701-01.png,2480,3416,0.725995
4,20180101-01.png,2481,3388,0.732290


In [18]:
df.loc[:,"ratio"] = df.apply(lambda row : text_ratio(row['img_path']), axis=1)

In [19]:
df['Date'] = [x[:8] for x in df['img_path']]
df['year'] = [x[:4] for x in df['Date']]
# df = df.drop('img_path', axis=1)
df.head(5)

,img_path,width,height,wh_ratio,ratio,Date,year
0,20161201-01.png,2480,3382,0.733294,0.156174,20161201,2016
1,20211201-01.png,2481,3383,0.733373,0.086030,20211201,2021
2,20120601-01.png,2480,3425,0.724088,0.247106,20120601,2012
3,20120701-01.png,2480,3416,0.725995,0.297922,20120701,2012
4,20180101-01.png,2481,3388,0.732290,0.261616,20180101,2018


In [ ]:
# testing with single image

image = cv2.imread('/Users/jessicacarr/Documents/J/github/magazine_text/covers/text/20110901-01.png')
image_width = image.shape[1]
image_height = image.shape[0]

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

plt.figure()

bounding_rectangles = []

for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    
    if area > 100:  
        x, y, w, h = cv2.boundingRect(contour)
        
        x_pct = (x / image_width) * 100
        y_pct = (y / image_height) * 100
        w_pct = (w / image_width) * 100
        h_pct = (h / image_height) * 100
        
        bounding_rectangles.append({
            "x": x, "x_pct": x_pct,
            "y": y, "y_pct": y_pct,
            "w": w, "w_pct": w_pct,
            "h": h, "h_pct": h_pct
        })
        
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        plt.plot([x, x + w, x + w, x, x], [y, y, y + h, y + h, y], label=f"Contour {i+1}")


# Convert BGR image to RGB for displaying with Matplotlib
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image_rgb)
plt.xlabel('X')
plt.ylabel('Y')
plt.title(f'Detected Contours on Original Image ({len(contours)} contours)')
plt.grid(False)

plt.show()

In [24]:
df.loc[:,"contours"] = df.apply(lambda row : get_contours_coordinates(row['img_path']), axis=1)
df.head(5)

,img_path,width,height,wh_ratio,ratio,Date,year,contours
0,20161201-01.png,2480,3382,0.733294,0.156174,20161201,2016,"[{'x': 1815, 'x_pct': 73.18548387096774, 'y': ..."
1,20211201-01.png,2481,3383,0.733373,0.086030,20211201,2021,"[{'x': 686, 'x_pct': 27.650141072148326, 'y': ..."
2,20120601-01.png,2480,3425,0.724088,0.247106,20120601,2012,"[{'x': 510, 'x_pct': 20.56451612903226, 'y': 3..."
3,20120701-01.png,2480,3416,0.725995,0.297922,20120701,2012,"[{'x': 1508, 'x_pct': 60.806451612903224, 'y':..."
4,20180101-01.png,2481,3388,0.732290,0.261616,20180101,2018,"[{'x': 1876, 'x_pct': 75.61467150342604, 'y': ..."


In [26]:
v = df.values.tolist()
c = df.columns.values.tolist()

#  commenting this line out or else it just prints it
# d = [dict(zip(c, x)) for x in v]

[{'img_path': '20161201-01.png',
  'width': 2480,
  'height': 3382,
  'wh_ratio': 0.7332939089296274,
  'ratio': 0.1561741716106141,
  'Date': '20161201',
  'year': '2016',
  'contours': [{'x': 1815,
    'x_pct': 73.18548387096774,
    'y': 2864,
    'y_pct': 84.6836191602602,
    'w': 573,
    'w_pct': 23.10483870967742,
    'h': 397,
    'h_pct': 11.738616203429924},
   {'x': 214,
    'x_pct': 8.629032258064516,
    'y': 2768,
    'y_pct': 81.84506209343584,
    'w': 1161,
    'w_pct': 46.814516129032256,
    'h': 84,
    'h_pct': 2.4837374334713185},
   {'x': 318,
    'x_pct': 12.822580645161292,
    'y': 2494,
    'y_pct': 73.74334713187463,
    'w': 953,
    'w_pct': 38.42741935483871,
    'h': 244,
    'h_pct': 7.214665878178593},
   {'x': 1991,
    'x_pct': 80.28225806451613,
    'y': 2350,
    'y_pct': 69.48551153163808,
    'w': 397,
    'w_pct': 16.008064516129032,
    'h': 397,
    'h_pct': 11.738616203429924},
   {'x': 527,
    'x_pct': 21.25,
    'y': 2232,
    'y_pct': 65

In [27]:
with open('mag_text.json', 'w') as fp:
    json.dump(d, fp)